In [1]:
import os
import urllib
import zipfile
import nltk
import numpy as np
import tensorflow as tf

In [2]:
EMBEDDING_DIMENSION=100 # Available dimensions for 6B data is 50, 100, 200, 300
data_directory = '/home/johannes/thesis_code/word_embeddings/'

glove_weights_file_path = '/home/johannes/thesis_code/word_embeddings/glove.6B.100d.txt'



In [3]:
PAD_TOKEN = 0

word2idx = { 'PAD': PAD_TOKEN } # dict so we can lookup indices for tokenising our text later from string to sequence of integers weights = []

weights=[]
with open(glove_weights_file_path,'r') as file:
    for index,line in enumerate(file):
        values = line.split()
        word = values[0]
        word_weights = np.asarray(values[1:], dtype=np.float32)
        word2idx[word] = index + 1
        weights.append(word_weights)
        if index + 1 == 40000:
            # Limit vocabulary to top 40k terms
            break


In [4]:
EMBEDDING_DIMENSION = len(weights[0])

In [5]:
weights.insert(0, np.random.randn(EMBEDDING_DIMENSION))
UNKNOWN_TOKEN=len(weights) 
word2idx['UNK'] = UNKNOWN_TOKEN 
weights.append(np.random.randn(EMBEDDING_DIMENSION))

In [6]:
weights = np.asarray(weights, dtype=np.float32)
VOCAB_SIZE=weights.shape[0]

In [7]:

features = {}
features['word_indices'] = nltk.word_tokenize('hello world') # ['hello', 'world']
features['word_indices'] = [word2idx.get(word, UNKNOWN_TOKEN) for word in features['word_indices']]


In [8]:
glove_weights_initializer = tf.constant_initializer(weights)
embedding_weights = tf.get_variable(
    name='embedding_weights', 
    shape=(VOCAB_SIZE, EMBEDDING_DIMENSION), 
    initializer=glove_weights_initializer,
    trainable=False)
#embedding = tf.nn.embedding_lookup(embedding_weights, features['word_indices'])

In [9]:
embedding_weights

<tf.Variable 'embedding_weights:0' shape=(40002, 100) dtype=float32_ref>

In [11]:
"""
******************
DONE WITH
WORD EMBEDDINGS??
******************
"""

'\n******************\nDONE WITH\nWORD EMBEDDINGS??\n******************\n'

In [12]:
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

k=24

In [43]:
x = tf.placeholder(tf.float32, shape=(EMBEDDING_DIMENSION)) # mnist data image of shape 28*28=784
h = tf.placeholder(tf.float32, shape=(EMBEDDING_DIMENSION))
y = tf.placeholder(tf.float32) # 0-9 digits recognition => 10 classes

In [44]:
W = tf.Variable(tf.math.add(tf.eye(EMBEDDING_DIMENSION),tf.random.normal([EMBEDDING_DIMENSION, EMBEDDING_DIMENSION],0,0.2)))

In [57]:
proj = tf.matmul(tf.expand_dims(x,0),W)
sim = tf.tensordot(proj, h, 1)
pred = tf.math.sigmoid(sim)

In [59]:
pred

<tf.Tensor 'Sigmoid_7:0' shape=(1,) dtype=float32>

In [60]:
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
init = tf.global_variables_initializer()